# Getting Started
#### To start this notebook, you must have a huggingface account and request access from Meta to use Llama 2.
https://huggingface.co/

#### In huggingface, create an access token
https://huggingface.co/docs/hub/security-tokens

#### Inside your home directory create .apikeys and inside that directory create a huggingface_api_key.txt and paste you access token inside the file. The path should look like the following.
path - /home/{your username}/.apikeys

#### Using the link below request access from Meta
https://huggingface.co/meta-llama/Llama-2-7b-hf

#### Once you recieve access from Meta inside terminal create a conda environment using
conda create --name {environment_name} python=3.10

#### Then Install ipykernel using
conda install ipykernel

#### To allow your environment to be used in the notebook run the following line and select your environment on the top right besides the debugging symbol
python -m ipykernel install --user --name={environment_name}

#### Go back to terminal and install all the packages with
pip install -r packages.txt

#### Edit the data set, test set, validation set, and prompt/prompt template function under Load Datasets with the path and you are good to go!
##### All imported data must be a csv. Csv must have at least 2 columns for output and test
##### Import the data inside the llama-recipes folder

#### If you are just loading a model run the first 4 cells then skip to the loading section

## Access Huggingface API Key

In [1]:
# Set cache directory and load Huggingface api key
import os

username = os.getenv('USER')
directory_path = os.path.join('/scratch', username)
output_dir = os.path.join('/scratch', username ,'llama-output')

# Set Huggingface cache directory to be on scratch drive
if os.path.exists(directory_path):
    hf_cache_dir = os.path.join(directory_path, 'hf_cache')
    if not os.path.exists(hf_cache_dir):
        os.mkdir(hf_cache_dir)
    print(f"Okay, using {hf_cache_dir} for huggingface cache. Models will be stored there.")
    assert os.path.exists(hf_cache_dir)
    os.environ['TRANSFORMERS_CACHE'] = f'/scratch/{username}/hf_cache/'
else:
    error_message = f"Are you sure you entered your username correctly? I couldn't find a directory {directory_path}."
    raise FileNotFoundError(error_message)

# Load Huggingface api key
api_key_loc = os.path.join('/home', username, '.apikeys', 'huggingface_api_key.txt')

if os.path.exists(api_key_loc):
    print('Huggingface API key loaded.')
    with open(api_key_loc, 'r') as api_key_file:
        huggingface_api_key = api_key_file.read().strip()  # Read and store the contents
else:
    error_message = f'Huggingface API key not found. You need to get an HF API key from the HF website and store it at {api_key_loc}.\n' \
                    'The API key will let you download models from Huggingface.'
    raise FileNotFoundError(error_message)

Okay, using /scratch/kwamea/hf_cache for huggingface cache. Models will be stored there.
Huggingface API key loaded.


In [2]:
import torch
import wandb
from transformers import AutoTokenizer, AutoModelForCausalLM, LlamaTokenizer, LlamaForCausalLM, GenerationConfig, pipeline, DataCollatorWithPadding, default_data_collator, Trainer, TrainingArguments
from langchain import PromptTemplate, LLMChain
from langchain.llms import HuggingFacePipeline
from configs import fsdp_config, train_config
from peft import get_peft_model, prepare_model_for_int8_training, PeftModelForCausalLM, LoraConfig, TaskType, prepare_model_for_int8_training, PeftModel
from utils.dataset_utils import get_preprocessed_dataset
from utils.train_utils import (
    train,
    freeze_transformer_layers,
    setup,
    setup_environ_flags,
    clear_gpu_cache,
    print_model_size,
)
from utils.config_utils import (
    update_config,
    generate_peft_config,
    generate_dataset_config,
)
from datasets import Dataset, load_dataset
from pathlib import Path
import sys
import csv
import random
import json
from configs.datasets import samsum_dataset, alpaca_dataset, grammar_dataset
from ft_datasets.utils import Concatenator
import huggingface_hub
from huggingface_hub import notebook_login, Repository, HfApi, create_repo, delete_repo
huggingface_hub.login(token = huggingface_api_key)

Token will not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /home/kwamea/.cache/huggingface/token
Login successful


## Load Llama 2 7B

In [3]:
tokenizer = AutoTokenizer.from_pretrained(
        "mistralai/Mistral-7B-v0.1", #mistralai/Mistral-7B-v0.1
        resume_download=True,
        cache_dir=os.path.join('/scratch', username),
        load_in_8bit=True if train_config.quantization else None,
        token=huggingface_api_key,
)

tokenizer.add_special_tokens(
    {
        "pad_token": "<PAD>",
    }
)

#the code will output "Error displaying widget: model not found" it is not an error just the code failing to create a loading bar

1

In [4]:
model = AutoModelForCausalLM.from_pretrained(
        "mistralai/Mistral-7B-v0.1", #meta-llama/Llama-2-7b-hf
        resume_download=True,
        load_in_8bit=True if train_config.quantization else None,
        device_map="auto" if train_config.quantization else None,
        cache_dir=os.path.join('/scratch', username),
        trust_remote_code = True,
        token=huggingface_api_key
)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

/home/kwamea/.conda/envs/llms_env/lib/python3.10/site-packages/transformers/utils/hub.py:374: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


## Load in your dataset

In [5]:
#add testset and rename current test set to validation set 

#edit file path to your unique dataset
dataset = load_dataset('csv', data_files='samsum-data/samsum-train.csv',split = 'train')
valset = load_dataset('csv', data_files='samsum-data/samsum-validation.csv',split = 'train')
testset = load_dataset('csv', data_files='samsum-data/samsum-test.csv',split = 'train')

shorten_percentage = 10

# Shorten the datasets
def shorten_dataset(dataset, percentage):
    num_samples = len(dataset)
    new_num_samples = int(num_samples * (percentage / 100.0))
    return dataset.shuffle(seed=42).select(list(range(new_num_samples)))

# Shorten each dataset
"""
dataset = shorten_dataset(dataset, shorten_percentage)
valset = shorten_dataset(valset, shorten_percentage)
testset = shorten_dataset(testset, shorten_percentage)
"""
#Edit the prompt to tell the model what to do including the variables from prompt.format
prompt = (
    f"Summarize this dialogue:\n{{dialog}}\n---\nSummary:{{summary}}\n"
)

#prompt for testing
test_prompt = (
    f"Summarize this dialogue:\n{{dialog}}\n---\nSummary:\n"
)

#edit the variables in prompt.format to match your data: essentially what you what the model to read
def apply_prompt_template(sample):
    return {
        "text": prompt.format(
            dialog = sample["dialogue"],
            summary = sample["summary"],
        )
    }

#Only include what you want the model to see during testing
def apply_prompt_template_TEST(sample):
    return {
        "text": test_prompt.format(
            dialog = sample["dialogue"],
        )
    }

data = dataset.map(apply_prompt_template, remove_columns=list(dataset.features))
val = valset.map(apply_prompt_template, remove_columns=list(valset.features))
test = testset.map(apply_prompt_template_TEST, remove_columns=list(testset.features))

train_dataset = data.map(
    lambda sample: tokenizer(sample["text"]),
    batched=True,
    remove_columns=list(data.features), 
).map(Concatenator(), batched=True)
val_dataset = val.map(
    lambda sample: tokenizer(sample["text"]),
    batched=True,
    remove_columns=list(val.features), 
).map(Concatenator(), batched=True)
test_dataset = test.map(
    lambda sample: tokenizer(sample["text"]),
    batched=True,
    remove_columns=list(test.features), 
).map(Concatenator(), batched=True)

Found cached dataset csv (/home/kwamea/.cache/huggingface/datasets/csv/default-04ed86c0369d6106/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (/home/kwamea/.cache/huggingface/datasets/csv/default-98e7740e620c6c6f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Found cached dataset csv (/home/kwamea/.cache/huggingface/datasets/csv/default-ed9c43e8e741b026/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1)
Loading cached processed dataset at /home/kwamea/.cache/huggingface/datasets/csv/default-04ed86c0369d6106/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-19ca57271f564c64.arrow
Loading cached processed dataset at /home/kwamea/.cache/huggingface/datasets/csv/default-98e7740e620c6c6f/0.0.0/6954658bab30a358235fa864b05cf819af0e179325c740e4bc853bcc7ec513e1/cache-538cc2747fc973be.arrow
Loading cached processed dataset at /home/kwamea/.cache/huggingface/datasets/csv/def

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/14732 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/818 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

Map:   0%|          | 0/819 [00:00<?, ? examples/s]

## Test the model before finetuning

In [6]:
#Edit eval_prompt to match your data
eval_prompt = """
Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
"""

model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")

model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



Summarize this dialog:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a puppy for my son.
B: That will make him so happy.
A: Yeah, we’ve discussed it many times. I think he’s ready now.
B: That’s good. Raising a dog is a tough issue. Like having a baby ;-) 
A: I'll get him one of those little dogs.
B: One that won't grow up too big;-)
A: And eat too much;-))
B: Do you know which one he would like?
A: Oh, yes, I took him there last Monday. He showed me one that he really liked.
B: I bet you had to drag him away.
A: He wanted to take it home right away ;-).
B: I wonder what he'll name it.
A: He said he’d name it after his dead hamster – Lemmy  - he's  a great Motorhead fan :-)))
---
Summary:
A: Hi Tom, are you busy tomorrow’s afternoon?
B: I’m pretty sure I am. What’s up?
A: Can you go with me to the animal shelter?.
B: What do you want to do?
A: I want to get a p

## Enables Parameter Efficient Finetuning (PEFT)

In [8]:
#reduces the parameters needed to train
model.train()
def create_peft_config(model):
    from peft import (
        get_peft_model,
        LoraConfig,
        TaskType,
        prepare_model_for_int8_training,
    )

    peft_config = LoraConfig(
        task_type=TaskType.CAUSAL_LM,
        inference_mode=False,
        r=8,
        lora_alpha=32,
        lora_dropout=0.05,
        bias= "none",
        target_modules = ["q_proj", "v_proj"],
    )
    kwargs = {
        'use_peft': True, 
        'peft_method': 'lora', 
        'quantization': True, 
        'use_fp16': True, 
        'model_name': os.path.join('/scratch', username, 'models--meta-llama--Llama-2-7b-hf/snapshots/6fdf2e60f86ff2481f2241aaee459f85b5b0bbb9'), 
        'output_dir': os.path.join('/scratch', username)
    }
    
    update_config((train_config, fsdp_config), **kwargs)
    
    model = prepare_model_for_int8_training(model)
    #peft_config = generate_peft_config(train_config, kwargs)
    model = get_peft_model(model, peft_config)
    model.print_trainable_parameters()
    return model, peft_config

# create peft config
model, lora_config = create_peft_config(model)

trainable params: 3,407,872 || all params: 7,245,139,968 || trainable%: 0.04703666202518836


In [9]:
torch.cuda.empty_cache()
from transformers import TrainerCallback
from contextlib import nullcontext
enable_profiler = False
#set up the configurations for training
config = {
    'lora_config': lora_config,
    'learning_rate': 1e-4,
    'num_train_epochs': 1, #2
    'gradient_accumulation_steps': 2,
    'per_device_train_batch_size': 2,
    'gradient_checkpointing': False,
}

# Set up profiler
if enable_profiler:
    wait, warmup, active, repeat = 1, 1, 2, 1
    total_steps = (wait + warmup + active) * (1 + repeat)
    schedule =  torch.profiler.schedule(wait=wait, warmup=warmup, active=active, repeat=repeat)
    profiler = torch.profiler.profile(
        schedule=schedule,
        on_trace_ready=torch.profiler.tensorboard_trace_handler(f"{output_dir}/logs/tensorboard"),
        record_shapes=True,
        profile_memory=True,
        with_stack=True)
    
    class ProfilerCallback(TrainerCallback):
        def __init__(self, profiler):
            self.profiler = profiler
            
        def on_step_end(self, *args, **kwargs):
            self.profiler.step()

    profiler_callback = ProfilerCallback(profiler)
else:
    profiler = nullcontext()

In [10]:
#create a weights and biases account and input the api key in below
#change the names of the project
wandb.init(project="tmp1", name="testMistral")

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: kwamea (finetuningnotebook). Use `wandb login --relogin` to force relogin


## Defines training arguments and trains the model

In [ ]:
#Plotted train loss
torch.cuda.empty_cache()
# Define training args
from transformers.integrations import WandbCallback

training_args = TrainingArguments(
    output_dir=output_dir,
    overwrite_output_dir=True,
    bf16=True, 
    #logging_dir=f"{output_dir}/logs",
    logging_strategy="steps",
    evaluation_strategy="steps",
    logging_steps=5,
    eval_steps=5,
    save_strategy="steps",
    optim="adamw_torch_fused",
    auto_find_batch_size = True, 
    max_steps=total_steps if enable_profiler else -1,
    **{k:v for k,v in config.items() if k != 'lora_config'},
    remove_unused_columns=False,
    do_eval=True,
    save_steps=10,
    save_total_limit=5,
    load_best_model_at_end=True,
    push_to_hub=False,  # Set this to False to avoid conflicts with W&B logging
    logging_dir=wandb.run.dir,
    report_to="wandb",
)
with profiler:
    # Create Trainer instance
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
        data_collator=default_data_collator,

        callbacks=[profiler_callback, WandbCallback()] if enable_profiler else [WandbCallback()],
    )
    
# Start training
trainer.train()

You are adding a <class 'transformers.integrations.integration_utils.WandbCallback'> to the callbacks of this Trainer, but there is already one. The currentlist of callbacks is
:DefaultFlowCallback
WandbCallback
`use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`...
/home/kwamea/.conda/envs/llms_env/lib/python3.10/site-packages/torch/utils/checkpoint.py:429: UserWarning: torch.utils.checkpoint: please pass in use_reentrant=True or use_reentrant=False explicitly. The default value of use_reentrant will be updated to be False in the future. To maintain current behavior, pass use_reentrant=True. It is recommended that you use use_reentrant=False. Refer to docs for more details on the differences between the two variants.
  warnings.warn(
/home/kwamea/.conda/envs/llms_env/lib/python3.10/site-packages/bitsandbytes/autograd/_functions.py:322: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"

Step,Training Loss,Validation Loss


## Save the model to output directory

In [ ]:
#saves the model
model.save_pretrained(output_dir)

#Saves model configurations
model.config.to_json_file(os.path.join(output_dir, "config.json"))

#saves PEFT config
peft_config = model.peft_config
json_file_path = os.path.join(output_dir, "peft_config.json")

# Custom serialization function for LoraConfig objects
def lora_config_serializer(obj):
    if isinstance(obj, LoraConfig):
        # Return a dictionary representation of the LoraConfig object
        return obj.__dict__
    raise TypeError("Type not serializable")

# Write the dictionary to the JSON file using the custom serializer
with open(json_file_path, "w") as json_file:
    json.dump(peft_config, json_file, default=lora_config_serializer, indent=4)

## Push to Huggingface Hub

In [ ]:
from transformers.tokenization_utils_base import PreTrainedTokenizerBase
from pathlib import Path
#https://huggingface.co/docs/hub/repositories-getting-started
api = HfApi()

repo_id = "andrk9/testMistral"

try:
    create_repo(repo_id)
except RepositoryExistsError:
    # If the repository already exists, catch the exception and update the repository
    print(f"Repository '{repo_id}' already exists. Updating...")

api.upload_folder(
    folder_path=output_dir,
    repo_id=repo_id,
    repo_type="model",
)

# Usage example after pushing the model to the hub
'''
repo_name = "your-username/your-repo-name"
loaded_tokenizer = AutoTokenizer.from_pretrained(repo_name)
loaded_model = AutoModelForSequenceClassification.from_pretrained(repo_name)
'''

## Test model on the same input as before

In [ ]:
#Tests the model on the sample input from before
model.eval()
with torch.no_grad():
    print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))


## Load Saved Model

In [ ]:
#load from huggingface
repo_id = "andrk9/testMistral"
model = PeftModelForCausalLM.from_pretrained(
        model,
        model_id=repo_id,
        load_in_8bit=True if train_config.quantization else None,
        device_map="auto" if train_config.quantization else None,
        cache_dir=os.path.join('/scratch', username),
        token=huggingface_api_key
)

In [ ]:
torch.cuda.empty_cache()
model = PeftModelForCausalLM.from_pretrained(model, output_dir)

In [ ]:
for _ in range(5):
    print("\nGenerated Output: ")
    random_idx = random.randint(0, len(test_dataset) - 1)
    eval_prompt = test['text'][random_idx]
    model_input = tokenizer(eval_prompt, return_tensors="pt").to("cuda")
    model.eval()
    with torch.no_grad():
        print(tokenizer.decode(model.generate(**model_input, max_new_tokens=100)[0], skip_special_tokens=True))